In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Installs
!pip install -q -U keras-tuner

     |████████████████████████████████| 71kB 3.6MB/s 


In [ ]:
# Imports
import gdown
import os
import json
import csv   
import cv2
import numpy as np
import pandas as pd
from PIL import Image
import scipy.misc
import sklearn
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
import tensorflow.keras.layers.experimental.preprocessing as keras_preproc
import kerastuner as kt
import keras
import io
import json
import pandas as pd
import tensorflow as tf
import sklearn
import matplotlib.pyplot as plt
import itertools
import numpy as np
import os, shutil
from datetime import datetime
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from collections import Counter
import random

In [ ]:
TUNING_TRAIN_IMAGES_PATH = 'tuning_train_images/'
TUNING_TRAIN_LABELS_PATH = 'tuning_train_data.csv'
VALIDATION_IMAGES_PATH = 'val_images/'
VALIDATION_LABELS_PATH = 'val_data.csv'

LABEL_TO_DISEASE_MAP_PATH = 'label_num_to_disease_map.json'
MODEL_CP_PATH = '/content/drive/MyDrive/ml_project/models/'
HYPERPARAMETERS_PATH = '/content/drive/MyDrive/ml_project/hyperparams'

N_CLASSES = 5

# Images
IMG_SIZE = 224
SIZE = (IMG_SIZE,IMG_SIZE)
INPUT_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

# Training
EPOCHS_TO_TRAIN = 30
BATCH_SIZE = 32

In [ ]:
os.chdir('/content/')
# Downloading data
if not os.path.exists(LABEL_TO_DISEASE_MAP_PATH):
  !wget files.brainfriz.com/ml_project/label_num_to_disease_map.json

if not os.path.exists(TUNING_TRAIN_IMAGES_PATH): 
  !wget files.brainfriz.com/ml_project/tuning_data.rar
  
if not os.path.exists('/content/drive/MyDrive/ml_project/'):
  !mkdir /content/drive/MyDrive/ml_project

if not os.path.exists(HYPERPARAMETERS_PATH):
  !mkdir /content/drive/MyDrive/ml_project/hyperparams

if not os.path.exists(MODEL_CP_PATH):
  !mkdir /content/drive/MyDrive/ml_project/models

--2021-03-24 10:30:49--  http://files.brainfriz.com/ml_project/tuning_data.rar
Resolving files.brainfriz.com (files.brainfriz.com)... 138.201.201.196
Connecting to files.brainfriz.com (files.brainfriz.com)|138.201.201.196|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://files.brainfriz.com/ml_project/tuning_data.rar [following]
--2021-03-24 10:30:51--  https://files.brainfriz.com/ml_project/tuning_data.rar
Connecting to files.brainfriz.com (files.brainfriz.com)|138.201.201.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4600500145 (4.3G) [application/x-rar-compressed]
Saving to: ‘tuning_data.rar’

tuning_data.rar     100%[===================>]   4.28G  11.0MB/s    in 6m 44s  

2021-03-24 10:37:35 (10.9 MB/s) - ‘tuning_data.rar’ saved [4600500145/4600500145]

mkdir: cannot create directory ‘/content/drive/MyDrive/ml_project’: No such file or directory
mkdir: cannot create directory ‘/content/drive/MyDrive/ml_

In [ ]:
!unrar x tuning_data.rar > /dev/null


In [ ]:
!rm -f tuning_train.rar

In [ ]:
# Dataset
train_data = pd.read_csv(TUNING_TRAIN_LABELS_PATH)
validation_data = pd.read_csv(VALIDATION_LABELS_PATH)

f = open(LABEL_TO_DISEASE_MAP_PATH)
label_class_name = json.load(f)
label_class_name = {int(k):v for k,v in label_class_name.items()}

train_data['class_name'] = train_data.label.map(label_class_name)
validation_data['class_name'] = validation_data.label.map(label_class_name)

train_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

proportion_data_used_for_tuning = 0.2

if proportion_data_used_for_tuning:
  train_indices = list(range(len(train_data)))
  random.shuffle(train_indices)
  train_indices = train_indices[:int(proportion_data_used_for_tuning*len(train_indices))]
  tuning_data = train_data.iloc[train_indices]

  print(Counter(tuning_data['class_name']))

train_gen = train_datagen.flow_from_dataframe(tuning_data,
                                directory = TUNING_TRAIN_IMAGES_PATH,
                                x_col = 'image_id',
                                y_col = 'class_name',
                                color_mode='rgb',
                                class_mode='categorical',
                                target_size = SIZE,
                                shuffle = True,
                                batch_size = BATCH_SIZE)

val_gen = test_datagen.flow_from_dataframe(validation_data,
                                directory = VALIDATION_IMAGES_PATH,
                                x_col = 'image_id',
                                y_col = 'class_name',
                                color_mode='rgb',
                                class_mode='categorical',
                                target_size = SIZE,
                                shuffle = False,
                                batch_size = BATCH_SIZE)

Counter({'Cassava Bacterial Blight (CBB)': 2604, 'Cassava Mosaic Disease (CMD)': 2588, 'Cassava Brown Streak Disease (CBSD)': 2575, 'Cassava Green Mottle (CGM)': 2524, 'Healthy': 2467})
Found 12758 validated image filenames belonging to 5 classes.
Found 1000 validated image filenames belonging to 5 classes.


In [ ]:
# Models definition

def getEfficientNetB0():
  return [
      # architecture
      tf.keras.applications.EfficientNetB0(
        include_top=True, weights=None, input_tensor=None,
        input_shape=INPUT_SHAPE, pooling=None, classes=N_CLASSES,
        classifier_activation='softmax', drop_connect_rate=0.4
      ),
      # preprocess_input
      tf.keras.applications.efficientnet.preprocess_input
  ]

def getResNet50(): 
  return [
      # architecture 
      tf.keras.applications.ResNet50(
        include_top=True, weights=None, input_tensor=None,
        input_shape=INPUT_SHAPE, pooling='avg', classes=N_CLASSES,
        classifier_activation='softmax'
      ),
      # preprocess_input
      tf.keras.applications.resnet_v2.preprocess_input
  ]

def getMobileNetV3Small(): 
  return [
      # architecture
      tf.keras.applications.MobileNetV3Small(
        input_shape=INPUT_SHAPE, alpha=1, minimalistic=True, include_top=True,
        weights=None, input_tensor=None, classes=5, pooling=None,
        dropout_rate=0, classifier_activation='softmax'
      ),
      # preprocess_input:
      tf.keras.applications.mobilenet_v3.preprocess_input
  ]


# Sourc: https://www.kaggle.com/marto24/keras-model-89-tta
def stacked_model_builder(hp):
  hp_learning_rate = hp.Float('learning_rate', 1e-05, 1, sampling='log')
  hp_label_smoothing = hp.Float('label_smoothing', 1e-09, 1e-02, sampling='log')


  hp_arch_drop_connect_rate = hp.Float('hp_arch_drop_connect_rate', 0.1, 0.9, step=0.1)
  hp_first_layers_excluded = hp.Int('hp_first_layers_excluded', 0, 1000, default=10)
  hp_dense_layer_1_units = hp.Int('hp_dense_layer_1_units', 64, 1024, default=256, step=64)
  hp_dense_layer_dropout = hp.Float('hp_dense_layer_dropout', 0.1, 0.9, default=0.5, step=0.1)
  hp_num_dense_layers = hp.Int('hp_num_dense_layers', 1, 5, default=2)

  model = Sequential()
  # initialize the model with input shape
  model.add(tf.keras.applications.EfficientNetB0(input_shape = (IMG_SIZE, IMG_SIZE, 3), include_top = False,
                           weights = 'imagenet',
                           drop_connect_rate=hp_arch_drop_connect_rate))
  for layer in model.layers[:hp_first_layers_excluded]:
     layer.trainable = False

  model.add(GlobalAveragePooling2D())
  model.add(Flatten())
  model.add(Dense(hp_dense_layer_1_units, activation = 'relu', bias_regularizer=tf.keras.regularizers.L1L2(l1=0.01, l2=0.001)))
  model.add(Dropout(hp_dense_layer_dropout))

  for i in range(1, hp_num_dense_layers):
    model.add(Dense(hp_dense_layer_1_units / 2**i, activation = 'relu', bias_regularizer=tf.keras.regularizers.L1L2(l1=0.01, l2=0.001))) # half the number of units from previous dense layer
    model.add(Dropout(hp_dense_layer_dropout))

  model.add(Dense(N_CLASSES, activation = 'softmax'))  
    
  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate, epsilon=0.1),
                loss=keras.losses.CategoricalCrossentropy(label_smoothing=hp_label_smoothing),
                metrics=['accuracy'])
    
  return model

def model_builder(hp):
  inputs = tf.keras.layers.Input(shape=INPUT_SHAPE)
  architecture, preprocess_input = getResNet50() # change architecture here
  #input_layer = preprocess_input(tf.keras.layers.Input(shape=INPUT_SHAPE)) #only for mobilenet change inputs to input_layer
  model = tf.keras.Model(inputs, architecture(inputs))
  
  hp_learning_rate = hp.Float('learning_rate', 1e-05, 1, sampling='log')
  hp_label_smoothing = hp.Float('label_smoothing', 1e-09, 1e-02, sampling='log')

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate, epsilon=0.1),
                loss=keras.losses.CategoricalCrossentropy(label_smoothing=hp_label_smoothing),
                metrics=['accuracy'])

  return model



In [ ]:
# Hyperparameter tuning
tuning_callbacks = [
  tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3),
  tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,patience=2, min_lr=0.00001),
]
  
tuner = kt.Hyperband(model_builder, # make sure to change the model in the model_builder function.
                     objective='val_accuracy',
                     max_epochs=15,
                     factor=3,
                     directory=HYPERPARAMETERS_PATH,
                     project_name='cassava')

tuner.search(train_gen, epochs=EPOCHS_TO_TRAIN, validation_data=val_gen, callbacks=tuning_callbacks)

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters()[0]
print(f"""
The hyperparameter search is complete. Optimal values: \n
learning_rate: {best_hps.get('learning_rate')}; \n
label_smoothing: {best_hps.get('label_smoothing')}; \n
""")

INFO:tensorflow:Reloading Oracle from existing project /content/drive/MyDrive/ml_project/hyperparams/cassava/oracle.json

Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
learning_rate     |0.54582           |?                 
label_smoothing   |0.20356           |?                 
tuner/epochs      |2                 |?                 
tuner/initial_e...|0                 |?                 
tuner/bracket     |1                 |?                 
tuner/round       |0                 |?                 

Epoch 1/2
  61/1994 [..............................] - ETA: 22:54 - loss: 17.4227 - accuracy: 0.1954

KeyboardInterrupt: ignored

In [ ]:
model = tuner.get_best_models(1)[0]
model_json = model.to_json()
with open(MODEL_CP_PATH+"tuning_model.json", "w+") as json_file:
  json_file.write(model_json)

model.save_weights(MODEL_CP_PATH+"tuning_model.h5")